# 9.1 概括数据
第 8 章介绍过如何把文本内容分解成 n-gram 模型，或者长度为 n 个单词的短语。从基本功 能上说，这可以用来确定一段文字中最常用的单词和短语。另外，还可用来从原文中提取 包含最常用的短语的句子，从而对原文进行合理的概括。

In [9]:
from collections import Counter
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import string



    
def cleanSentence(sentence):
    sentence = sentence.split(" ")
    sentence = [word.strip(string.punctuation+string.whitespace)for word in sentence]
    sentence = [word for word in sentence if len(word)>1 or 
                    (word.lower()=='a'or word.lower()=='i')]
    return sentence 
    
def cleanInput(content):
    content = re.sub("\n|[[\d+\]]"," ",content)
    content = bytes(content,"UTF-8")

    content = content.decode("ascii", "ignore")
    sentences = content.split('. ')
    return [cleanSentence(sentence) for sentence in sentences]


def getNgramsFromSentence(content,n):
    output = []
    for i in range(len(content)-n+1):
        output.append(content[i:i+n])
    return output
    
def getNgrams(content,n):
    content = cleanInput(content)
    ngrams = Counter()
    ngrams_list = []
    for sentence in content:
        newNgrams = [' '.join(ngram) for ngram in getNgramsFromSentence(sentence, 2)]
        ngrams_list.extend(newNgrams)
        ngrams.update(newNgrams)
    return ngrams




In [10]:
content = str(
           urlopen('http://pythonscraping.com/files/inaugurationSpeech.txt')
             .read(), 'utf-8')

ngrams = getNgrams(content, 2)

print(ngrams)

Counter({'of the': 212, 'in the': 62, 'to the': 60, 'by the': 41, 'the Constitution': 32, 'of our': 29, 'to be': 26, 'from the': 24, 'the people': 23, 'that the': 23, 'of a': 22, 'and the': 22, 'may be': 19, 'of their': 19, 'of that': 18, 'for the': 16, 'of its': 16, 'of power': 16, 'have been': 16, 'is the': 16, 'with the': 15, 'it is': 15, 'all the': 14, 'which they': 13, 'has been': 13, 'power to': 12, 'the Government': 12, 'the power': 11, 'that of': 11, 'as to': 11, 'their own': 11, 'spirit of': 11, 'the Executive': 11, 'which the': 10, 'which it': 10, 'upon the': 10, 'can be': 10, 'the United': 10, 'United States': 10, 'the same': 10, 'as the': 10, 'part of': 10, 'the character': 10, 'of liberty': 10, 'of my': 9, 'of all': 9, 'of any': 9, 'the President': 9, 'the whole': 9, 'to a': 8, 'of those': 8, 'which our': 8, 'the great': 8, 'and that': 8, 'and of': 8, 'the legislative': 8, 'It is': 8, 'the States': 8, 'would be': 8, 'in their': 8, 'character of': 8, 'there is': 8, 'the pub

/var/folders/95/0b9jlx8d4dgdp4pw27fwrd440000gn/T/ipykernel_2137/1656675707.py:19: FutureWarning: Possible nested set at position 3
  content = re.sub("\n|[[\d+\]]"," ",content)


In [ ]:
'''停用词的使用'''

# 9.2 马尔可夫模型
你可能听说过马尔可夫文本生成器。它们因为两种用途而非常受欢迎:娱乐，比如用在
That can be my next tweet! 应用中;用于生成逼真的垃圾邮件来愚弄检测系统。


**马尔可夫模型常用于分析大量的随机事件，其中一
个离散事件发生之后，另一个离散事件会以一定的概率发生**

In [15]:
'''还用前面例子里分析的威廉 • 亨利 • 哈里森的就职演讲内容，我们可以写出下面的代码，
基于文本结构生成任意长度(下面示例中链长为 100)的马尔可夫链。'''
from urllib.request import urlopen
from random import randint 
def wordListSum(wordList):
    sum = 0
    for word,value in wordList.items():
        sum += value
    return sum 

def retrieveRandomWord(wordList):
    randIndex = randint(1,wordListSum(wordList))
    for word,value in wordList.items():
        randIndex -= value
        if randIndex <= 0:
            return word 

def buildWordDict(text):
    # 剔除换行符和引号
    text = text.replace('\n', ' ') 
    text = text.replace('"', '')

    # 保证每个标点符号都被当作一个"单词"
    # 这样就不会被剔除，而是会保留在马尔可夫链中 
    punctuation = [',','.',';',':']
    for symbol in punctuation:
        text = text.replace(symbol, ' {} '.format(symbol))
    
    words = text.split(' ')
    # 过滤空单词
    words = [word for word in words if word !=""]
    wordDict = {}
    for i in range(1, len(words)):
        if words[i-1] not in wordDict:
                # Create a new dictionary for this word
            wordDict[words[i-1]] = {}
        if words[i] not in wordDict[words[i-1]]:
            wordDict[words[i-1]][words[i]] = 0
        wordDict[words[i-1]][words[i]] += 1
    return wordDict



In [16]:
text = str(urlopen('http://pythonscraping.com/files/inaugurationSpeech.txt').read(), 'utf-8')
wordDict = buildWordDict(text)

In [24]:
'''生成链长为100的马尔可夫链'''
length = 20 
chain = ['I']
for i in range(0,length):
    newWord = retrieveRandomWord(wordDict[chain[-1]])
    chain.append(newWord)
print(' '.join(chain))

I refer to a provision so happily subsists with entire control of the United States accept a selection of a vestige



**维基百科六度分隔:终结篇**



广度优先搜索算法的思路是优先搜寻直接连接到起始页的所有链接(而不是找到一个链接
就纵向深入搜索)。如果这些链接不包含目标页面(你想要找的词条)，就对第二层链接 (通过一个中间页面链接到起始页)进行搜索。这个过程不断重复，直到达到搜索深度限
制(本例中使用的层数限制是 6 层)或者找到目标页面为止。

In [49]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
          user='root', passwd='2022mysql', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute('USE scraping')


0

In [50]:

def getUrl(pageId):
    cur.execute('SELECT url FROM wikipedia_pages WHERE id = %s', (int(pageId)))
    return cur.fetchone()[0]

def getLinks(fromPageId):
    cur.execute('SELECT toPageId FROM wikipedia_links WHERE fromPageId = %s', (int(fromPageId)))
    if cur.rowcount == 0:
        return []
    return [x[0] for x in cur.fetchall()]
def searchBreadth(targetPageId, paths=[[1]]):
    newPaths = []
    for path in paths:
        links = getLinks(path[-1])
        for link in links:
            if link == targetPageId:
                return path + [link]
            else:
                newPaths.append(path+[link])
    return searchBreadth(targetPageId, newPaths)


In [52]:
nodes = getLinks(1)
targetPageId = 3000
pageIds = searchBreadth(targetPageId)


RecursionError: maximum recursion depth exceeded

In [ ]:
for pageId in pageIds:
    print(getUrl(pageId))

# 运用nltk做相关

In [1]:
from nltk import word_tokenize
from nltk import Text

In [2]:
tokens = word_tokenize("here is some text")
text = Text(tokens)
text

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/danyow/nltk_data'
    - '/Users/danyow/envs/myenvs/nltk_data'
    - '/Users/danyow/envs/myenvs/share/nltk_data'
    - '/Users/danyow/envs/myenvs/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [3]:
from nltk.book import * 


*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.


LookupError: 
**********************************************************************
  Resource [93mgutenberg[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('gutenberg')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/gutenberg[0m

  Searched in:
    - '/Users/danyow/nltk_data'
    - '/Users/danyow/envs/myenvs/nltk_data'
    - '/Users/danyow/envs/myenvs/share/nltk_data'
    - '/Users/danyow/envs/myenvs/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
